# Lecture 3. Performance guidelines.

In this notebook we will the most important guidelines when programming code for NVIDIA GPU cards.

We will consider the following aspects of optimizing CUDA kernels:
- memory access patterns: *memory coalescing* for the best throughput,
- control flow: how code branching affects the performance,
- multiprocessor occupancy: experimenting with different block sizes,
- instruction-level optimizations: avoiding data type conversion, using floating point *intrisincs*.

In [1]:
! pip install --upgrade --force-reinstall git+https://github.com/pjarosik/ius-2021-gpu-short-course.git

  Cloning https://github.com/pjarosik/ius-2021-gpu-short-course.git to /tmp/pip-req-build-pytg2ac3
  Running command git clone -q https://github.com/pjarosik/ius-2021-gpu-short-course.git /tmp/pip-req-build-pytg2ac3
  Created wheel for gpu-short-course: filename=gpu_short_course-0.0.1-py3-none-any.whl size=2924 sha256=9c78b40d750dc3ddd1d47d13ac10f34157d417e85ac8aba3820cb4dcb4dc7821
  Stored in directory: /tmp/pip-ephem-wheel-cache-okxpyrfb/wheels/4f/07/fc/9537d8ac1b84ce9cde4db4fcebd10fd77e93ea2c18fcb8a656
Successfully built gpu-short-course
  Attempting uninstall: gpu-short-course
    Found existing installation: gpu-short-course 0.0.1
    Uninstalling gpu-short-course-0.0.1:
      Successfully uninstalled gpu-short-course-0.0.1


## Exercise 3.1. Memory access patterns.

According to [CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html):
> For devices of compute capability 6.0 or higher, the requirements can be summarized quite easily: the concurrent accesses of the threads of a warp will coalesce into a number of transactions equal to the number of 32-byte transactions necessary to service all of the threads of the warp.

This means that the number of useful memory accesses done by our kernel, and thus its performance, largely depends on the memory access pattern it does.

Our goal is to implement a GPU kernel that only loads **useful** data from global memory that will then be used in the calculations. We can achieve this with **coalesced memory accesses**.

To achive coalesced memory accesses in our kernel, we need to meet the following conditions:
- the number of threads per block is a multiple of 32 threads,
- sequential threads in a warp access memory that is sequential.


For example, our baseline `add_vectors_gpu` and `convolve_gpu` implementations satisfy the above conditions:
- the number of threads per block was equal 256,
- adjacent threads were reading adjacent memory areas, e.g. thread `i` read the `a[i]` and `b[i]`, and thread `i+1` read `a[i+1]` and `b[i+1]`.  


We will discuss below what are the reasons for both of the conditions.

### Exercise 3.1.1. Impact of misaligned accesses.

According to [CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html):
> The number of threads per block should be a multiple of 32 threads, because this provides optimal computing efficiency and facilitates coalescing.

Recall that warp reads global memory by using a sequence **32-byte** segments transactions.

Note that, according to [CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html):
> Memory allocated through the CUDA Runtime API (...) is guaranteed to be aligned to at least 256 bytes.

This means that if the block size is a multiple of the warp size, each block will load only its own data chunk from memory. 


#### Example

As an example, we will consider here:
- `add_vectors_gpu` function,
- block size = 13. 

Now let's take a look what memory accesses will be performed by thread block 0, 1, etc.

**Block 0**

- reads memory area [0, 52), size: 52 bytes (13 x 4-byte floats)

```
[ Segment 0 (32 bytes) ][ Segment 1 (32 bytes) ][ Segment 2 (32 bytes) ] ...
[       block 0 data (52 bytes)       ]
```

- This requires 2 x 32-byte transfers. 
- However only 52 bytes will be used (81%). 

**Block 1**

- Reads memory area [52, 104), size: 52 bytes.
```
[ Segment 0 (32 bytes) ][ Segment 1 (32 bytes) ][ Segment 2 (32 bytes) ] ...
                                       [       block 1 data (52 bytes)       ]
```
- This requires 3 x 32-byte transfers.
- However only 52 bytes will be used (54%). 


**And so on...**


As we can see, we are transfer (theoretically) a large amount of unnecessary data.

Let's see if there is any observable performance difference between scripts using 256 and 261 threads in a **block**:

In [5]:
%%writefile 3_1_1_aligned.py

from numba import cuda
import math
import numpy as np
import gpu_short_course.tests

block_size = 256


@cuda.jit
def add_vectors_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    result[i] = a[i] + b[i]


def add_vectors_gpu(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()

gpu_short_course.tests.benchmark_add_vectors(add_vectors_gpu)

Overwriting 3_1_1_aligned.py


In [16]:
! nvprof --trace gpu python 3_1_1_aligned.py

Benchmarking the function, please wait...
==16609== NVPROF is profiling process 16609, command: python 3_1_1_aligned.py
Benchmark result: 
Average processing time: 0.0486 seconds (+/- 0.3719), median: 0.0111
==16609== Profiling application: python 3_1_1_aligned.py
==16609== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   55.89%  419.12ms       300  1.3971ms  1.2828ms  2.8090ms  [CUDA memcpy DtoH]
                   40.77%  305.75ms       200  1.5287ms  1.3798ms  3.2402ms  [CUDA memcpy HtoD]
                    3.33%  24.986ms       100  249.86us  247.83us  271.22us  cudapy::__main__::add_vectors_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
No API activities were profiled.


In [11]:
%%writefile 3_1_1_misaligned.py

from numba import cuda
import math
import numpy as np
import gpu_short_course.tests

block_size = 261


@cuda.jit
def add_vectors_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    result[i] = a[i] + b[i]


def add_vectors_gpu(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()

gpu_short_course.tests.benchmark_add_vectors(add_vectors_gpu)

Overwriting 3_1_1_misaligned.py


In [14]:
! nvprof --trace gpu python 3_1_1_misaligned.py

Benchmarking the function, please wait...
==16528== NVPROF is profiling process 16528, command: python 3_1_1_misaligned.py
Benchmark result: 
Average processing time: 0.0192 seconds (+/- 0.0729), median: 0.0118
==16528== Profiling application: python 3_1_1_misaligned.py
==16528== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   57.59%  444.93ms       300  1.4831ms  1.2941ms  2.6090ms  [CUDA memcpy DtoH]
                   39.11%  302.13ms       200  1.5107ms  1.3879ms  2.5283ms  [CUDA memcpy HtoD]
                    3.30%  25.455ms       100  254.55us  250.71us  273.11us  cudapy::__main__::add_vectors_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
No API activities were profiled.


The difference in performance of the aligned and misaligned versions is rather minor (on some devices even negligible).

Why? 

In this particular case, adjacent warps **reuse the cached data** their neighbors fetched. 

Anyway, setting a block size to a multiple of warp size, might be a good rule of thumb: it facilitates coalescing, and (as we will discuss later), helps to avoid wasting multiprocessor computation time on under-populated warps.

### Exercise 3.1.2. Impact of strided accesses.

A non-unit-strided global memory accesses may impact effective memory bandwidth. 

We say that GPU kernel performs a unit-strided memory access, if threads with successive identifiers read the data from successive memory areas, in other words, the following access pattern is respected:

```
x = data[(some custom offset) + threadIdx.x]
```

When using the data access notation for a multidimensional array, make sure that the last axis is addressed using `threadIdx.x`:

```
x = data[(other dimensions...), threadIdx.x]
```


The degradation in performance can be especially apparent when working with multidimensional arrays - the choice of the axis, along which choosing a specific operation is performed, can affect effective bandwidth.

#### Example

Let's consider a 1D convolution along one of the axes of a 2D array.

```
         axis 1
     ---------------
x = [[0,  1,  2,  3], |
     [4,  5,  6,  7], | axis 0
     [8,  9, 10, 11]] |

h = [1, 1]

```

NumPy stores arrays in row-major order, so the above array is actually kept in computer's memory as a following 1D array:

```
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 
```

Let's consider doing convolution along axis 0 and 1.

**Convolve along axis 1**:

```
         axis 1
     ---------------
x = [[0,  1,  2,  3]  *  [1, 1] = [0,  1,  3,  5] 
     [4,  5,  6,  7], *  [1, 1] = [4,  9, 11, 13]
     [8,  9, 10, 11]] *  [1, 1] = [8, 17, 19, 21]
```

For the first output row:

```
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]  
    [1]      y = [0]
    [1, 1]       [1]
       [1, 1]    [3]
          [1, 1] [5]

```

`y[0]` `y[1]`, `y[2]` and `y[3]` are computed by threads with `threadIdx.x` equal `0`, `1`, `2` and `3`, respectively.

**Convolve along axis 0**:

```
x = [[0,  1,   2,   3]  | 
     [4,  5,   6,   7], | axis 0
     [8,  9,  10,  11]] | 
      *   *    *    *
     [1] [1]  [1]  [1]
     [1] [1]  [1]  [1]
   y  =   =    =    =
    [ 0] [ 1] [ 2] [ 3]
    [ 4] [ 6] [ 8] [10]
    [12] [14] [16] [18]
```

For the first output column:

```
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]  
    [1]                                    y = [ 0]
    [1,          1]                            [ 4]
                [1,         1]                 [12]
```

`y[0]` `y[1]`, and `y[2]` are computed by threads with `threadIdx.x` equal `0`, `1` and `2` respectively.

As we can see in the above example, the stride is much larger for the convolution along axis 0. Will it impact the bandwidth?

In [2]:
%%writefile 3_1_2_convolve_strided_access.py
import math
import numpy as np
from numba import cuda, float32
import cupy as cp
import gpu_short_course

@cuda.jit
def convolve_axis0_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    j = cuda.blockIdx.y*cuda.blockDim.y + cuda.threadIdx.y

    N = len(h)
    o = int(math.ceil(N/2)-1)
    HEIGHT = x.shape[0]
    WIDTH = x.shape[1]
    if i >= HEIGHT or j >= WIDTH:
        return
    
    value = float32(0.0)
    for k in range(N):
        l = i + o - k
        if l >= 0 and l < HEIGHT:

            ## --- Get data along the second (1) axis.
            value += x[l, j] * h[k]
            
    y[i, j] = value
    
    
def convolve_axis0_gpu(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block = (32, 32)
    height, width = x.shape
    block_h, block_w = block
    grid = (math.ceil(width/block_w), 
            math.ceil(height/block_h))
    convolve_axis0_gpu_kernel[grid, block](y, x, h)
    return y.copy_to_host()


@cuda.jit
def convolve_axis1_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    j = cuda.blockIdx.y*cuda.blockDim.y + cuda.threadIdx.y

    N = len(h)
    o = int(math.ceil(N/2)-1)
    
    HEIGHT = x.shape[0]
    WIDTH = x.shape[1]
    
    if i >= WIDTH or j >= HEIGHT:
        return
    
    value = float32(0.0)
    for k in range(N):
        l = i+o-k
        if l >= 0 and l < WIDTH:
            
            ## --- Get data along the first (0) axis.
            value += x[j, l]*h[k]
            
    y[j, i] = value
    
    
def convolve_axis1_gpu(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block = (32, 32)
    height, width = x.shape
    block_h, block_w = block
    grid = (math.ceil(width/block_w), 
            math.ceil(height/block_h))
    convolve_axis1_gpu_kernel[grid, block](y, x, h)
    return y.copy_to_host()

gpu_short_course.convolve_2d_input(convolve_axis0_gpu, axis=0)
gpu_short_course.convolve_2d_input(convolve_axis1_gpu, axis=1)

Overwriting 3_1_2_convolve_strided_access.py


In [3]:
! python 3_1_2_convolve_strided_access.py --mode test

All tests passed.
All tests passed.


In [8]:
! nvprof --trace gpu python 3_1_2_convolve_strided_access.py --mode benchmark quiet=1

Benchmarking, please wait...
==61799== NVPROF is profiling process 61799, command: python 3_1_2_convolve_strided_access.py --mode benchmark quiet=1
Benchmarking, please wait...
==61799== Profiling application: python 3_1_2_convolve_strided_access.py --mode benchmark quiet=1
==61799== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.41%  6.13775s       100  61.378ms  53.455ms  78.364ms  cudapy::__main__::convolve_axis0_gpu_kernel$241(Array<float, int=2, C, mutable, aligned>, Array<float, int=2, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                   31.94%  3.29992s       100  32.999ms  26.750ms  39.088ms  cudapy::__main__::convolve_axis1_gpu_kernel$242(Array<float, int=2, C, mutable, aligned>, Array<float, int=2, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                    5.70%  588.47ms       600  980.79us     800ns  2.6401ms  [CUDA memcpy DtoH]
                 

On my GPU (Nvidia GeForce MX250), convolution along axis 1 takes much less time than along axis 0.

We can use profiler metrics to verify what memory access efficiency for both cases we have:

In [11]:
! nvprof --metrics gld_efficiency,gst_efficiency python 3_1_2_convolve_strided_access.py --mode benchmark n=1 quiet=1 2>&1 | grep -v "^="

Benchmarking, please wait...
Benchmarking, please wait...
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "GeForce MX250 (0)"
    Kernel: cudapy::__main__::convolve_axis0_gpu_kernel$241(Array<float, int=2, C, mutable, aligned>, Array<float, int=2, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
          1                            gld_efficiency             Global Memory Load Efficiency      12.50%      12.50%      12.50%
          1                            gst_efficiency            Global Memory Store Efficiency      12.50%      12.50%      12.50%
    Kernel: cudapy::__main__::convolve_axis1_gpu_kernel$242(Array<float, int=2, C, mutable, aligned>, Array<float, int=2, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
          1                            gld_efficiency             Global Memory Load Efficiency      70.05%      70.05%      70.05%
          1    

# Exercise 3.2. Control flow: how code branching affects the performance.

Due to SIMT architecture of the CUDA multiprocessors, it is recommended to avoid different paths within the same warp.

According to [CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html):
> Flow control instructions (if, switch, do, for, while) can significantly affect the instruction throughput by causing threads of the same warp to diverge; that is, to follow different execution paths. If this happens, the different execution paths must be executed separately; this increases the total number of instructions executed for this warp.

### Example

Let's implement the following function:

```
y[i] = r[i]*a[i] + b[i]
```

where `r[i] = i mod 8`.

We can implement it in one of the two ways:
1. directly by definition (see `add_vectors_mod8_kernel`),
2. by doing a sequence of `if ... elif ... elif ... else` blocks (see `add_vectors_mod8_branches_kernel`).

In [35]:
%%writefile 3_2_control_flow.py

from numba import cuda
import math
import numpy as np
import gpu_short_course.tests


block_size = 256


@cuda.jit
def add_vectors_mod8_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    r = i % 8 + 1
    result[i] = r*a[i] + b[i]


def add_vectors_mod8(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_mod8_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()


@cuda.jit
def add_vectors_mod8_branches_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    if i % 8 == 0:
        result[i] = a[i] + b[i]
    elif i % 8 == 1:
        result[i] = 2*a[i] + b[i]
    elif i % 8 == 2:
        result[i] = 3*a[i] + b[i]
    elif i % 8 == 3:
        result[i] = 4*a[i] + b[i]
    elif i % 8 == 4:
        result[i] = 5*a[i] + b[i]
    elif i % 8 == 5:
        result[i] = 6*a[i] + b[i]
    elif i % 8 == 6:
        result[i] = 7*a[i] + b[i]
    elif i % 8 == 7:
        result[i] = 8*a[i] + b[i]


def add_vectors_mod8_branches(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_mod8_branches_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()


gpu_short_course.tests.benchmark_add_vectors(add_vectors_mod8)
gpu_short_course.tests.benchmark_add_vectors(add_vectors_mod8_branches)

Overwriting 3_2_control_flow.py


Let's check how much time does it take to execute each of the kernel:

In [36]:
! nvprof --trace gpu python 3_2_control_flow.py

Benchmarking the function, please wait...
==69197== NVPROF is profiling process 69197, command: python 3_2_control_flow.py
Benchmark result: 
Average processing time: 0.0178 seconds (+/- 0.0545), median: 0.0117
Benchmarking the function, please wait...
Benchmark result: 
Average processing time: 0.0273 seconds (+/- 0.0253), median: 0.0239
==69197== Profiling application: python 3_2_control_flow.py
==69197== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   44.96%  1.33802s       100  13.380ms  1.4619ms  35.427ms  cudapy::__main__::add_vectors_mod8_branches_kernel$242(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                   31.22%  929.08ms       600  1.5485ms  1.2940ms  2.6280ms  [CUDA memcpy DtoH]
                   21.21%  631.13ms       400  1.5778ms  1.4059ms  2.6370ms  [CUDA memcpy HtoD]
                    2.60%  77.462ms    

Let's also measure `branch_efficiency` metric defined as a:
> Ratio of non-divergent branches to total branches expressed as percentage.

In [37]:
! nvprof --metrics branch_efficiency python 3_2_control_flow.py

Benchmarking the function, please wait...
==69273== NVPROF is profiling process 69273, command: python 3_2_control_flow.py
Benchmark result: 
Average processing time: 0.0211 seconds (+/- 0.0468), median: 0.0151
Benchmarking the function, please wait...
Benchmark result: 
Average processing time: 0.0305 seconds (+/- 0.0210), median: 0.0257
==69273== Profiling application: python 3_2_control_flow.py
==69273== Profiling result:
==69273== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "GeForce MX250 (0)"
    Kernel: cudapy::__main__::add_vectors_mod8_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
        100                         branch_efficiency                         Branch Efficiency     100.00%     100.00%     100.00%
    Kernel: cudapy::__main__::add_vectors_mod8_branches_kernel$242(Arra

Of course, the above example has been artificially complicated just to show the effect of complex kernel logic on the kernel's performance.

# Exercise 3.3. Multiprocessor occupancy: thread block size.

Recall that:
> The number of threads per block should be a **multiple of 32 threads**, because this provides optimal computing efficiency and facilitates coalescing.

[CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html) also gives some other suggestions how to choose the proper number of threads per block:

> There are many such factors involved in selecting block size, and inevitably some experimentation is required. However, a few rules of thumb should be followed:
> 1. Threads per block should be **a multiple of warp size** to avoid wasting computation on under-populated warps and to facilitate coalescing.
> 2. A **minimum of 64 threads** per block should be used, and only if there are multiple concurrent blocks per multiprocessor.
> 3. Between **128 and 256 threads** per block is a good initial range for experimentation with different block sizes.
> 4. Use several smaller thread blocks rather than one large thread block per multiprocessor if latency affects performance. This is particularly beneficial to kernels that frequently call __syncthreads().

### Example

Let's mesure `add_vectors`' occupancy for a different number of threads:


In [39]:
%%writefile 3_3_occupancy_16.py

from numba import cuda
import math
import numpy as np
import gpu_short_course.tests

block_size = 16


@cuda.jit
def add_vectors_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    result[i] = a[i] + b[i]


def add_vectors_gpu(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()


gpu_short_course.tests.benchmark_add_vectors(add_vectors_gpu)

Writing 3_3_occupancy_16.py


In [43]:
! nvprof --trace gpu python 3_3_occupancy_16.py

Benchmarking the function, please wait...
==71869== NVPROF is profiling process 71869, command: python 3_3_occupancy_16.py
Benchmark result: 
Average processing time: 0.0172 seconds (+/- 0.0486), median: 0.0120
==71869== Profiling application: python 3_3_occupancy_16.py
==71869== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.77%  455.71ms       300  1.5190ms  1.2940ms  2.6364ms  [CUDA memcpy DtoH]
                   38.61%  327.26ms       200  1.6363ms  1.4478ms  2.6613ms  [CUDA memcpy HtoD]
                    7.62%  64.599ms       100  645.99us  634.43us  794.43us  cudapy::__main__::add_vectors_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
No API activities were profiled.
Benchmarking the function, please wait...
==71904== NVPROF is profiling process 71904, command: python 3_3_occupancy_16.py
Benchmark result: 
Average 

According to NVIDIA documentation, `achieved_occupancy` measures:
> Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor.

In [44]:
! nvprof --metrics achieved_occupancy python 3_3_occupancy_16.py

Benchmarking the function, please wait...
==71957== NVPROF is profiling process 71957, command: python 3_3_occupancy_16.py
Benchmark result: 
Average processing time: 0.0252 seconds (+/- 0.0450), median: 0.0189
==71957== Profiling application: python 3_3_occupancy_16.py
==71957== Profiling result:
==71957== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "GeForce MX250 (0)"
    Kernel: cudapy::__main__::add_vectors_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
        100                        achieved_occupancy                        Achieved Occupancy    0.302625    0.338981    0.325882


In [46]:
%%writefile 3_3_occupancy_256.py

from numba import cuda
import math
import numpy as np
import gpu_short_course.tests

block_size = 256


@cuda.jit
def add_vectors_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    result[i] = a[i] + b[i]


def add_vectors_gpu(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()


gpu_short_course.tests.benchmark_add_vectors(add_vectors_gpu)

Writing 3_3_occupancy_256.py


In [47]:
! nvprof --trace gpu python 3_3_occupancy_256.py
! nvprof --metrics achieved_occupancy python 3_3_occupancy_256.py

Benchmarking the function, please wait...
==72133== NVPROF is profiling process 72133, command: python 3_3_occupancy_256.py
Benchmark result: 
Average processing time: 0.0177 seconds (+/- 0.0553), median: 0.0118
==72133== Profiling application: python 3_3_occupancy_256.py
==72133== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.10%  456.25ms       300  1.5208ms  1.2900ms  2.6553ms  [CUDA memcpy DtoH]
                   38.58%  313.75ms       200  1.5687ms  1.3848ms  2.6183ms  [CUDA memcpy HtoD]
                    5.32%  43.290ms       100  432.90us  247.65us  921.06us  cudapy::__main__::add_vectors_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
No API activities were profiled.
Benchmarking the function, please wait...
==72168== NVPROF is profiling process 72168, command: python 3_3_occupancy_256.py
Benchmark result: 
Avera

In [50]:
%%writefile 3_3_occupancy_1024.py

from numba import cuda
import math
import numpy as np
import gpu_short_course.tests

block_size = 1024


@cuda.jit
def add_vectors_kernel(result, a, b):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(result):
        return
    result[i] = a[i] + b[i]


def add_vectors_gpu(a, b):
    result = cuda.device_array(shape=a.shape, dtype=a.dtype)
    grid_size = math.ceil(len(a)/block_size)
    add_vectors_kernel[grid_size, block_size](result, a, b)
    return result.copy_to_host()

gpu_short_course.tests.benchmark_add_vectors(add_vectors_gpu)

Overwriting 3_3_occupancy_1024.py


In [51]:
! nvprof --trace gpu python 3_3_occupancy_1024.py
! nvprof --metrics achieved_occupancy python 3_3_occupancy_1024.py

Benchmarking the function, please wait...
==72367== NVPROF is profiling process 72367, command: python 3_3_occupancy_1024.py
Benchmark result: 
Average processing time: 0.0183 seconds (+/- 0.0486), median: 0.0132
==72367== Profiling application: python 3_3_occupancy_1024.py
==72367== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.29%  488.78ms       300  1.6293ms  1.2964ms  4.2568ms  [CUDA memcpy DtoH]
                   38.43%  333.72ms       200  1.6686ms  1.4115ms  3.5513ms  [CUDA memcpy HtoD]
                    5.28%  45.814ms       100  458.14us  249.70us  1.1106ms  cudapy::__main__::add_vectors_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
No API activities were profiled.
Benchmarking the function, please wait...
==72402== NVPROF is profiling process 72402, command: python 3_3_occupancy_1024.py
Benchmark result: 
Av

Finding the right number of threads per block requires some experimentation, but generally 128 or 256 threads is a good starting point.

# Exercise 3.4. Instruction-level optimizations.

This chapter covers the following:

1. Impact of the data type selection and conversion on the kernel's performance.
2. Floating-point intrinsics.

## Exercise 3.4.1. Data types.

When implementing a CUDA GPU kernel, keep the following in mind:

- the choice of the input data type may affect the kernel performance,
- data type conversion in kernel implementation may affect kernel performance.

Let's do some comparison of  `float32` and `float64` data, based on an example of a one-dimensional convolution.

First, let's recall our baseline implementation of convolution operator.

NOTE:
- our benchmark function generates `float32` input data,
- we used in the kernel implementation the `float32` keyword to enforce the proper data type of `value` variable. 

In [1]:
%%writefile 3_4_1_convolve_float32.py
import math
import numpy as np
from numba import cuda, float32
from gpu_short_course.tests import benchmark_convolve


@cuda.jit
def convolve_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(y):
        return
    M, N = len(x), len(h)
    
    o = int(math.ceil(N/2)-1)

    value = float32(0.0)
    for j in range(N):
        k = i+o-j
        if k >= 0 and k < M:
            value += x[k]*h[j]
    y[i] = value
    

def convolve_gpu(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = 256
    grid_size = math.ceil(len(y)/block_size)
    convolve_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()


benchmark_convolve(convolve_gpu, dtype=np.float32)

Overwriting 3_4_1_convolve_float32.py


In [3]:
! nvprof --trace gpu python 3_4_1_convolve_float32.py

Benchmarking the function, please wait...
==78701== NVPROF is profiling process 78701, command: python 3_4_1_convolve_float32.py
Benchmark result: 
Average processing time: 0.0362 seconds (+/- 0.0585), median: 0.0302
==78701== Profiling application: python 3_4_1_convolve_float32.py
==78701== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   82.47%  2.26756s       100  22.676ms  17.149ms  30.855ms  cudapy::__main__::convolve_gpu_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                   11.61%  319.13ms       300  1.0638ms  1.1520us  2.6456ms  [CUDA memcpy DtoH]
                    5.92%  162.88ms       200  814.39us     928ns  2.5892ms  [CUDA memcpy HtoD]
No API activities were profiled.


Now let's get rid of the `float32` keyword on line `17` and see if it has any effect on performance.

In [75]:
%%writefile 3_4_1_convolve_float32_and_float64.py
import math
import numpy as np
from numba import cuda, float32
from gpu_short_course.tests import benchmark_convolve


@cuda.jit
def convolve_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(y):
        return
    M, N = len(x), len(h)
    
    o = int(math.ceil(N/2)-1)

    value = 0.0
    for j in range(N):
        k = i+o-j
        if k >= 0 and k < M:
            value += x[k]*h[j]
    y[i] = value
    

def convolve_gpu(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = 256
    grid_size = math.ceil(len(y)/block_size)
    convolve_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()


benchmark_convolve(convolve_gpu, dtype=np.float32)

Writing 3_4_1_convolve_float32_and_float64.py


In [4]:
! nvprof --trace gpu python 3_4_1_convolve_float32_and_float64.py

Benchmarking the function, please wait...
==78730== NVPROF is profiling process 78730, command: python 3_4_1_convolve_float32_and_float64.py
Benchmark result: 
Average processing time: 0.0519 seconds (+/- 0.0569), median: 0.0473
==78730== Profiling application: python 3_4_1_convolve_float32_and_float64.py
==78730== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   89.74%  3.89248s       100  38.925ms  26.496ms  50.494ms  cudapy::__main__::convolve_gpu_kernel$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                    6.74%  292.19ms       300  973.97us  1.1200us  2.6160ms  [CUDA memcpy DtoH]
                    3.52%  152.71ms       200  763.56us     960ns  2.5823ms  [CUDA memcpy HtoD]
No API activities were profiled.


The processing may increase, because 0.0 is a float64, and we are doing promotion from float32 to float64, i.e.:

`value += (float64)(x[k]*h[j])`

then we downgrade from float64 to float32:

`y[i] = (float32)value`.

(note: the above may vary between different GPUs)

Lets check what results we will get when we will use only `float64` values in computations.

In [6]:
%%writefile 3_4_1_convolve_float64.py

import math
import numpy as np
from numba import cuda, float32
from gpu_short_course.tests import benchmark_convolve


@cuda.jit
def convolve_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(y):
        return
    M, N = len(x), len(h)
    
    o = int(math.ceil(N/2)-1)

    value = 0.0
    for j in range(N):
        k = i+o-j
        if k >= 0 and k < M:
            value += x[k]*h[j]
    y[i] = value
    

def convolve_gpu(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = 256
    grid_size = math.ceil(len(y)/block_size)
    convolve_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()


benchmark_convolve(convolve_gpu, dtype=np.float64)

Overwriting 3_4_1_convolve_float64.py


In [7]:
! nvprof --trace gpu python 3_4_1_convolve_float64.py

Benchmarking the function, please wait...
==79096== NVPROF is profiling process 79096, command: python 3_4_1_convolve_float64.py
Benchmark result: 
Average processing time: 0.0373 seconds (+/- 0.0595), median: 0.0310
==79096== Profiling application: python 3_4_1_convolve_float64.py
==79096== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   66.93%  1.90606s       100  19.061ms  17.431ms  22.869ms  cudapy::__main__::convolve_gpu_kernel$241(Array<double, int=1, C, mutable, aligned>, Array<double, int=1, C, mutable, aligned>, Array<double, int=1, C, mutable, aligned>)
                   21.80%  620.79ms       300  2.0693ms  1.1520us  5.1669ms  [CUDA memcpy DtoH]
                   11.28%  321.16ms       200  1.6058ms     992ns  5.2513ms  [CUDA memcpy HtoD]
No API activities were profiled.


The above results may differ between different GPU cards.

## Exercise 3.4.2. Floating point intrisincs.

CUDA Math API provides a set of intrinsic functions, specialized to carry out various calculations. Sometimes, using an intrisinc function explicitly in your code may improve its performance.

A complete list of intrinsic functions for CUDA C/C++ is available [here](https://docs.nvidia.com/cuda/cuda-math-api/).

A complete list of floating-point intrinsics in Numba is avalable [here](https://numba.pydata.org/numba-doc/latest/cuda-reference/kernel.html#floating-point-intrinsics). 

Note: At the stage of optimizing the machine code, the compiler may decide to use the intrinsic function (the same or similar), regardless of whether we used it in our implementation or not. Still, if you want to be sure that the intrinsic function is used, we should call it explicitly.

Let's check if using `cuda.fma` intrinsic in our `convolution` gives us any improvement:

In [8]:
%%writefile 3_4_2_convolve_intrinsics.py
import math
from numba import cuda, float32
import numpy as np
import gpu_short_course.tests
import cupy as cp


@cuda.jit
def convolve_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(y):
        return

    M, N = len(x), len(h)
    o = int(math.ceil(N/2)-1)
    
    value = float32(0.0)
    for j in range(N):
        k = i + o - j
        if k >= 0 and k < M:
            value += x[k]*h[j]
    y[i] = value


def convolve(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = 256
    grid_size = math.ceil(len(y)/block_size)
    convolve_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()


@cuda.jit
def convolve_fma_gpu_kernel(y, x, h):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if i >= len(y):
        return

    M, N = len(x), len(h)
    o = int(math.ceil(N/2)-1)
    
    value = float32(0.0)
    for j in range(N):
        k = i + o - j
        if k >= 0 and k < M:
            value = cuda.fma(x[k], h[j], value)
    y[i] = value


def convolve_fma(x, h):
    y = cuda.device_array(x.shape, dtype=x.dtype)
    block_size = 256
    grid_size = math.ceil(len(y)/block_size)
    convolve_fma_gpu_kernel[grid_size, block_size](y, x, h)
    return y.copy_to_host()


gpu_short_course.tests.benchmark_convolve(convolve_fma)
gpu_short_course.tests.benchmark_convolve(convolve)

Writing 3_4_2_convolve_intrinsics.py


In [9]:
!nvprof --trace gpu python 3_4_1_convolve_gpu.py

Benchmarking the function, please wait...
==80097== NVPROF is profiling process 80097, command: python 3_4_1_convolve_gpu.py
Benchmark result: 
Average processing time: 0.0324 seconds (+/- 0.0586), median: 0.0257
Benchmarking the function, please wait...
Benchmark result: 
Average processing time: 0.0280 seconds (+/- 0.0099), median: 0.0263
==80097== Profiling application: python 3_4_1_convolve_gpu.py
==80097== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.44%  1.94581s       100  19.458ms  17.212ms  28.853ms  cudapy::__main__::convolve_gpu_kernel$242(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>)
                   39.87%  1.91840s       100  19.184ms  17.029ms  27.130ms  cudapy::__main__::convolve_gpu_kernel_fma$241(Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, aligned>, Array<float, int=1, C, mutable, a